<a href="https://colab.research.google.com/github/gleidsonnunes/scripts/blob/master/Narrador_XTTS_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install TTS==0.21.1 gradio pymupdf ebooklib pydub bs4
!apt-get update && apt-get install -y ffmpeg


In [ ]:
from google.colab import files
uploaded = files.upload()

# Espera-se que o arquivo enviado seja renomeado como 'voz_clonada_sample.wav'
import shutil
for name in uploaded:
    if name.endswith(".mp3") or name.endswith(".wav"):
        shutil.move(name, "voz_clonada_sample.wav")


In [ ]:
import gradio as gr
from TTS.api import TTS
import fitz  # PyMuPDF
from ebooklib import epub
from bs4 import BeautifulSoup
import os
from pydub import AudioSegment
import torch # Import torch

os.makedirs("audios", exist_ok=True)

SPEAKER_WAV = "voz_clonada_sample.wav"
OUTPUT_DIR = "audios"

# Aceita os termos automaticamente (Coqui)
os.environ["COQUI_TOS_AGREED"] = "1"

# Allowlist XttsConfig and XttsAudioConfig for torch.load
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.models.xtts import XttsAudioConfig # Import XttsAudioConfig
torch.serialization.add_safe_globals([XttsConfig, XttsAudioConfig, BaseDatasetConfig])


tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=False, gpu=False)

emotions = ["neutral", "happy", "sad", "angry", "excited", "sleepy", "whispering", "shouting"]

def extract_text(file):
    ext = os.path.splitext(file.name)[-1].lower()
    text = ""
    if ext == ".pdf":
        doc = fitz.open(stream=file.read(), filetype="pdf")
        for page in doc:
            text += page.get_text()
    elif ext == ".epub":
        book = epub.read_epub(file.name)
        for item in book.get_items():
            if item.get_type() == epub.EpubHtml:
                soup = BeautifulSoup(item.get_content(), 'html.parser')
                text += soup.get_text()
    return text.strip()

def narrar(texto, emocao):
    wav_path = os.path.join(OUTPUT_DIR, "narracao.wav")
    mp3_path = os.path.join(OUTPUT_DIR, "narracao.mp3")

    tts.tts_to_file(
        text=texto,
        speaker_wav=SPEAKER_WAV,
        language="pt",
        emotion=emocao,
        file_path=wav_path
    )

    audio = AudioSegment.from_wav(wav_path)
    audio.export(mp3_path, format="mp3")

    return wav_path, mp3_path

with gr.Blocks() as demo:
    gr.Markdown("## Narrador XTTS com sua voz preferida 🎤")

    with gr.Row():
        arquivo = gr.File(label="Carregar PDF ou EPUB", file_types=[".pdf", ".epub"])
        emocao = gr.Dropdown(emotions, label="Emoção", value="neutral")

    texto_extraido = gr.Textbox(label="Texto extraído", lines=10)
    carregar_btn = gr.Button("Extrair texto")

    carregar_btn.click(fn=extract_text, inputs=arquivo, outputs=texto_extraido)

    with gr.Row():
        narrar_btn = gr.Button("Narrar")
        audio_saida = gr.Audio(label="Ouvir áudio")
        download_mp3 = gr.File(label="Baixar em MP3")

    narrar_btn.click(fn=narrar, inputs=[texto_extraido, emocao], outputs=[audio_saida, download_mp3])

demo.launch(share=True)

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL TTS.config.shared_configs.BaseDatasetConfig was not an allowed global by default. Please use `torch.serialization.add_safe_globals([BaseDatasetConfig])` or the `torch.serialization.safe_globals([BaseDatasetConfig])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about